**1**. (10 points)

Consider an eager (i.e. non-lazy) form of the map function that takes a list `xs` to another list `ys` by applying the same function `f` to each member of `xs`. Write a recursive version of map `rmap` and test it on the two simple cases shown below.

In [1]:
list(map(lambda x: x + x, []))

[]

In [2]:
list(map(lambda x: x + x, [1,2,3]))

[2, 4, 6]

In [3]:
def rmap(f, xs):
    """Recursive version of map."""
    
    if xs == []:
        return []
    else:
        return [f(xs[0])] + rmap(f, xs[1:])

In [4]:
rmap(lambda x: x + x, [])

[]

In [5]:
rmap(lambda x: x + x, [1,2,3])

[2, 4, 6]

**2**. (25 points)

- Using the `requests` package, download berries 1, 2, and 3 from `https://pokeapi.co/api/v2/berry` in JSON format (5 points)
- Convert to a `pandas` DataFrame (5 points)
- Create a new DataFrame that only retains numeric columns and the name column (5 points)
- Show only rows wheere the name begins with the letter `c` (5 points)
- Convert to a numpy array (excluding `name`) and standardize so each **row** has mean 0 and standard deviation 1 (5 points)

- The necessary package imports are made for you

In [6]:
import requests
import numpy as np
import pandas as pd
from pandas.api.types import is_numeric_dtype

In [7]:
berries = []
for i in range(1, 4):
    berry = requests.get('https://pokeapi.co/api/v2/berry/{}/'.format(i)).json()
    berries.append(berry)

In [8]:
berries_df = pd.DataFrame(berries)

In [9]:
idx = np.nonzero([is_numeric_dtype(x) for x in berries_df.dtypes])

In [10]:
idx = np.r_[idx[0], [berries_df.columns.tolist().index('name')]]

In [11]:
df = berries_df.iloc[:, idx]

In [12]:
df[df.name.str.startswith('c')]

,growth_time,id,max_harvest,natural_gift_power,size,smoothness,soil_dryness,name
0,3,1,5,60,20,25,15,cheri
1,3,2,5,60,80,25,15,chesto


In [13]:
xs = df.iloc[:, :-1].values

In [14]:
(xs - xs.mean(axis=1)[:, None])/xs.std(axis=1)[:, None]

array([[-0.81514658, -0.92081372, -0.70947943,  2.19636716,  0.08302419,
         0.34719206, -0.18114368],
       [-0.84377741, -0.87872677, -0.77387869,  1.14833612,  1.84732332,
        -0.07489149, -0.42438509],
       [-0.92270547, -0.92270547, -0.82333719,  1.90929056,  0.91560774,
         0.17034563, -0.32649578]])

**3**. (25 points)

We have provided an SQLite3 database with 3 tables `dog`, `treat` and `dog_treat`. The `dog_treat` table is a linker table showing which dog ate which treat. 

- Show a table of ALL dogs and the treats with calories that they ate with column names `dog`, `treat`, `calorie`. A dog that did not eat any treats should still be present in the table (15 points)
- Using a common table expression, show a table with two columns `dog` and `total_calories` where only dogs that have eaten more than 500 calories are displayed (5 points)

In [15]:
%load_ext sql

In [16]:
%sql sqlite:///pets.db

'Connected: @pets.db'

In [17]:
%sql SELECT * FROM sqlite_master

 * sqlite:///pets.db
Done.


type,name,tbl_name,rootpage,sql
table,dog,dog,2,"CREATE TABLE dog ( ""index"" BIGINT, dog_id BIGINT, name TEXT, age BIGINT)"
index,ix_dog_index,dog,3,"CREATE INDEX ix_dog_index ON dog (""index"")"
table,treat,treat,4,"CREATE TABLE treat ( ""index"" BIGINT, treat_id BIGINT, name TEXT, calories BIGINT)"
index,ix_treat_index,treat,5,"CREATE INDEX ix_treat_index ON treat (""index"")"
table,dog_treat,dog_treat,6,"CREATE TABLE dog_treat ( ""index"" BIGINT, treat_id BIGINT, dog_id BIGINT)"
index,ix_dog_treat_index,dog_treat,7,"CREATE INDEX ix_dog_treat_index ON dog_treat (""index"")"


In [18]:
%%sql

SELECT dog.name as dog, treat.name as treat, treat.calories
FROM dog
LEFT JOIN dog_treat
ON dog.dog_id = dog_treat.dog_id
LEFT JOIN treat
ON dog_treat.treat_id = treat.treat_id

 * sqlite:///pets.db
Done.


dog,treat,calories
Fido,Chewies,100
Fido,Bones,500
Roger,Bones,500
Woof,Bones,500
Woof,Raw Meat,450
Lassie,None,None


In [19]:
%%sql

with t AS
(SELECT dog.name as dog, treat.name as treat, treat.calories
FROM dog
LEFT JOIN dog_treat
ON dog.dog_id = dog_treat.dog_id
LEFT JOIN treat
ON dog_treat.treat_id = treat.treat_id)
SELECT dog, SUM(calories) as total_calories
FROM t
GROUP BY dog
HAVING total_calories > 500

 * sqlite:///pets.db
Done.


dog,total_calories
Fido,600
Woof,950


**4**. (40 points)

You want to evaluate whether a liner, quadratic or cubic polynomial is the best model for a set of data using leave-one-out cross-validation (LOOCV) and the mean squared error as evaluation metric. Write a function named `loocv` that takes the predictor variable `x`, the outcome variable `y`, a list of of degrees of polynomial models to be evaluated, and an evaluation function and returns the best model found by LOOCV. For example, you would call the function like this `loocv(x, y, [1,2,3], mse)` where `mse` is of course a function that returns the mean squared error.

- Use the `x` and `y` variables provided
- Do not use any packages except for the standard library and `numpy`
- Code snipets for fitting and estimation of polynomials is provided

In [20]:
import numpy as np

In [21]:
x = np.load('x.npy')
y = np.load('y.npy')

In [22]:
coeffs = np.polyfit(x, y, 2)

In [23]:
ypred = np.polyval(coeffs, x)

In [24]:
x

array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9])

In [25]:
x[np.ones(10).astype('bool')]

array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9])

In [26]:
def loocv(x, y, degrees, metric):
    """Performs LOOCV to find best polynomial modle."""
    
    n = len(x)
    losses = []
    for d in degrees:
        loss = 0
        for i in range(n):
            idx = np.ones(n).astype('bool')
            idx[i] = False
            xx = x[idx]
            yy = y[idx]
            coeffs = np.polyfit(xx, yy, d)    
            ypred = np.polyval(coeffs, x[i])
            loss += metric(y[i], ypred)
        losses.append(loss)
    k = np.argmin(losses)
    return degrees[k]

In [27]:
def mse(y, ypred):
    """Returns MSE between y and ypred."""
    
    return np.mean((y - ypred)**2)

In [28]:
loocv(x, y, [1,2,3, 4], mse)

2